In [1]:
bb = b"These are the times that try men's souls."

In [2]:
memv = memoryview(bb)

In [3]:
memv

In [4]:
memv[0], chr(memv[0]), hex(memv[0])

(84, 'T', '0x54')

In [5]:
memv[:10]

In [6]:
chr(memv[:10][0])

'T'

In [7]:
memv.readonly

True

In [8]:
ba = bytearray(b"If the facts don't fit the theory, change the facts.")

In [9]:
mutable1 = memoryview(ba)
mutable2 = mutable1[:10]

In [10]:
mutable1[0] = ord('A')

In [11]:
chr(mutable2[0])

'A'

In [12]:
ba[:1]

bytearray(b'A')

In [13]:
import numpy as np

In [14]:
np_mv = memoryview(np.ones((10, 20, 30)))

In [15]:
np_mv.ndim

3

In [16]:
np_mv.shape

(10, 20, 30)

In [17]:
np_mv.strides  # C contiguous in memory
               # last dim is cont. (row cont.)

(4800, 240, 8)

In [18]:
np_mv.itemsize, np_mv.format

(8, 'd')

In [19]:
# np structured dtype
dt = np.dtype([('a', np.int8), ('b', np.complex128)])

In [20]:
dt

dtype([('a', 'i1'), ('b', '<c16')])

In [21]:
structured_mv = memoryview(np.empty((10, ), dtype=dt))

In [22]:
structured_mv.format

'T{b:a:=Zd:b:}'

In [23]:
np.empty((10, ), dtype=dt)[0] = (1, 3+4j)

In [24]:
%load_ext Cython

In [25]:
%%cython
def summer(double[:] mv):
    """Sums its argument's contents."""
    cdef double d, ss = 0.0
    for d in mv:
        ss += d
    return ss

In [26]:
arr = np.ones((10**6,), dtype=np.double)

In [27]:
from array import array

In [28]:
a = array('d', [1] * 10**6)

In [29]:
%timeit summer(arr)

1 loops, best of 3: 233 ms per loop


In [30]:
%timeit summer(a)

1 loops, best of 3: 232 ms per loop


In [31]:
%timeit np.sum(arr)

1000 loops, best of 3: 474 µs per loop


In [32]:
%timeit np.sum(a)

1000 loops, best of 3: 489 µs per loop


In [33]:
%%cython
def summer_cstyle(double[:] mv):
    cdef:
        double ss = 0.0
        int i, N
    N = mv.shape[0]
    for i in range(N):
        ss += mv[i]
    return ss

In [34]:
%timeit summer_cstyle(arr)

1000 loops, best of 3: 846 µs per loop


In [35]:
%timeit summer_cstyle(a)

1000 loops, best of 3: 840 µs per loop


In [36]:
%%cython
from cython cimport boundscheck, wraparound

# @boundscheck(False)
# @wraparound(False)
def summer_unsafe(double[:] mv):
    cdef:
        double ss = 0.0
        int i, N
    N = mv.shape[0]
    with boundscheck(False), wraparound(False):
        for i in range(N):
            ss += mv[i]
    return ss    

In [37]:
%timeit summer_unsafe(arr)

1000 loops, best of 3: 840 µs per loop


In [38]:
%timeit summer_unsafe(a)

1000 loops, best of 3: 839 µs per loop


Declare a C-conti buffer

In [39]:
%%cython
from __future__ import print_function
import numpy as np

cdef float[:, ::1] c_contig_mv
c_contig_mv = np.ones((3, 4), dtype=np.float32)

try:
    c_contig_mv = np.ones((3, 4), dtype=np.float32, order='F')
except ValueError as e:
    print("Cannot assign Fortan-order array:", e)
    
try:
    arr = np.ones((3, 4), dtype=np.float32, order='F')
    c_contig_mv = arr[:, ::2]
except ValueError as e:
    print("Cannot assign strided array:", e)    

Cannot assign Fortan-order array: ndarray is not C-contiguous
Cannot assign strided array: ndarray is not C-contiguous


Declare Fortran-conti buffer

In [40]:
%%cython
import numpy as np
cdef double[::1, :] f_contig_mv = np.ones((3, 4), dtype=np.float64, order='F')

In [41]:
%%cython
from cython cimport boundscheck, wraparound

# @boundscheck(False)
# @wraparound(False)
def summer_unsafe_c_cont(double[::1] mv):
    cdef:
        double ss = 0.0
        int i, N
    N = mv.shape[0]
    with boundscheck(False), wraparound(False):
        for i in range(N):
            ss += mv[i]
    return ss    

In [42]:
arr = np.ones((10**6,), dtype=np.double)

In [43]:
%timeit summer_unsafe_c_cont(a)

1000 loops, best of 3: 833 µs per loop


In [44]:
%timeit summer_unsafe_c_cont(arr)

1000 loops, best of 3: 832 µs per loop


In [45]:
%%cython
from __future__ import print_function
cimport numpy as np_c
import numpy as np

DTYPE = np.int
ctypedef np_c.int_t DTYPE_t

def use_np_array(np_c.ndarray[DTYPE_t, ndim=1] my_arr):
    print(np.sum(my_arr))

In [46]:
import numpy as np

In [47]:
use_np_array(np.arange(10, dtype=DTYPE))

45


### Using buffer with C arrays

In [48]:
%%cython
from __future__ import print_function

cdef int a[3][5][7]
cdef int[:, :, ::1] mv = a

mv[...] = <int>0  # initialize mv's value to 0
print(mv[0][0][0])

0
